In [ ]:
# Run this ONCE in your terminal (NOT in the script) to install dependencies:
# pip install requests==2.32.3 ibm-watsonx-ai PyPDF2

import json
import requests
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai import Credentials
from PyPDF2 import PdfReader
from tkinter import Tk, filedialog

# ✅ Your credentials
API_KEY = "hzPuIY9ArkA9AUC53ejUmKX-ZVx7CjZ9HXqYwZpHtLub"
PROJECT_ID = "175e0636-b309-488b-a296-fc7a95fbd42d"
MODEL_ID = "ibm/granite-13b-instruct-v2"

# ✅ Setup Watsonx Connection
credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=API_KEY
)

model = ModelInference(
    model_id=MODEL_ID,
    credentials=credentials,
    project_id=PROJECT_ID
)

print("✅ Watsonx Granite 13B model connected successfully!")

# ✅ Use File Dialog to select PDF file
print("📂 Please select a PDF medical report to analyze.")
Tk().withdraw()  # Hide the root window
file_path = filedialog.askopenfilename(filetypes=[("PDF files", "*.pdf")])



if not file_path:
    print("❌ No file selected. Exiting.")
    exit()

# ✅ Extract text from PDF
try:
    reader = PdfReader(file_path)
    raw_text = ""
    for page in reader.pages:
        raw_text += page.extract_text() or ""
except Exception as e:
    print(f"❌ Error reading PDF: {e}")
    exit()

if not raw_text.strip():
    print("❌ No text could be extracted from the PDF.")
    exit()

print("\n📄 Extracted Text Preview:")
print(raw_text[:500])  # preview first 500 characters

# ✅ Build a clean summarization prompt
prompt = f"""
You are a professional medical assistant.

Here is a patient's medical report:

{raw_text}

Please summarize the key medical history, recent complaints, lab findings, medications, and doctor's notes in simple bullet points for the doctor to review quickly.
"""

# ✅ Generate Summary
try:
    response = model.generate(prompt)
    summary = response['results'][0]['generated_text']
except Exception as e:
    print(f"❌ Error during summary generation: {e}")
    exit()

# ✅ Display
print("\n✅ Generated Summary:")
print(summary)

# ✅ Doctor asks a follow-up question
doctor_question = input("\n👨‍⚕️ Enter your question about the patient's report: ")

# ✅ Build prompt for QnA
qa_prompt = f"""
You are a professional medical assistant.

Here is the summarized medical report:

{summary}

The doctor asks: {doctor_question}

Answer clearly and concisely based on the report only.
"""

# ✅ Generate AI Answer
try:
    response = model.generate(qa_prompt)
    answer = response['results'][0]['generated_text']
except Exception as e:
    print(f"❌ Error during QnA generation: {e}")
    exit()

# ✅ Display
print("\n✅ AI Answer:")
print(answer)


✅ Watsonx Granite 13B model connected successfully!
📂 Please select a PDF medical report to analyze.
